In [2]:
import pandas as pd
import numpy as np

In [3]:
#execute nlp2 before this to get new sets
x_train = pd.read_pickle('./train-both.pkl')
x_test = pd.read_pickle('./test-both.pkl')
y_train = pd.read_pickle('./y_train-both.pkl')
y_test = pd.read_pickle('./y_test-both.pkl')
test_pred = pd.read_pickle('./pred-test-both-lgbm.pkl')
train_pred = pd.read_pickle('./pred-train-both-lgbm.pkl')

In [4]:
print(test_pred.shape,train_pred.shape)

(6829, 20) (38697, 20)


In [41]:
x_train.columns

Index([          'text',        'HADM_ID', 'ADMISSION_DAYS',         'GENDER',
                  'AGE',            50801,            50802,            50803,
                  50804,            50805,
       ...
                  51512,            51513,            51514,            51515,
                  51516,            51517,            51518,            51519,
                  51529,            51532],
      dtype='object', length=485)

In [44]:
#x_train = x_train.drop(['HADM_ID'],axis=1)
print(x_train.shape)
print(x_train.columns)

(38697, 484)
Index([          'text', 'ADMISSION_DAYS',         'GENDER',            'AGE',
                  50801,            50802,            50803,            50804,
                  50805,            50806,
       ...
                  51512,            51513,            51514,            51515,
                  51516,            51517,            51518,            51519,
                  51529,            51532],
      dtype='object', length=484)


In [5]:
y_predtest_new = np.subtract(y_test,test_pred)
y_predtrain_new = np.subtract(y_train,train_pred)

In [6]:
from gensim.models import Word2Vec
model = Word2Vec.load('model1500-10-3.bin')

In [6]:
WordVectorz=dict(zip(model.wv.index2word,model.wv.vectors))
class AverageEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim =100 # because we use 100 embedding points 

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


In [7]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier,XGBRegressor
from lightgbm import LGBMClassifier,LGBMRegressor
from catboost import CatBoostClassifier,CatBoostRegressor
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
from sklearn.multioutput import MultiOutputRegressor
#model1 = CatBoostClassifier(iterations=1000, task_type="GPU",devices='0:1',learning_rate=0.1)
#d_train = lgb.Dataset(x_train, label=y_train)
#clf_multilabel = OneVsRestClassifier(LGBMClassifier(iterations=100))
#pipe1=Pipeline([("wordVectz",AverageEmbeddingVectorizer(WordVectorz)),("multilabel",MultiOutputRegressor(
#    XGBRegressor(iterations=1000,learning_rate=0.15,n_jobs=-1)))])

clf_multiregressor = MultiOutputRegressor(LGBMRegressor(iterations=1000,learning_rate=0.1))
#clf_multiregressor.fit(x_train,y_predtrain_new)

In [29]:
x_train.shape

(38697, 485)

In [37]:
y_predtrain_new.shape

(38697, 20)

In [46]:

x_train = np.reshape(x_train,(x_train.shape[0],1,x_train.shape[1]))
y_predtrain_new = np.reshape(y_train,(y_predtrain_new.shape[0],1,y_predtrain_new
                                      .shape[1]))

ValueError: Must pass 2-d input

In [47]:
import keras
from keras.layers import Dense,Conv1D
from keras.models import Sequential

model = Sequential()
model.add(Dense(x_train.shape[1]))
model.add(Dense(10))
model.add(Dense(y_predtrain_new.shape[1],activation='sigmoid'))

#model.summary()


In [56]:
'''import tensorflow as tf
from sklearn.metrics import roc_auc_score

def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)'''
model.compile(metrics=['binary_accuracy'],optimizer='adam',loss='binary_crossentropy')
#model.summary()

In [61]:
model.fit(x_train.values,y_predtrain_new.values,epochs=3,batch_size=10000)


Epoch 1/3
38697/38697 [==============================] - 1s 13us/step - loss: -52390.0206 - binary_accuracy: 0.0367
Epoch 2/3
38697/38697 [==============================] - 0s 12us/step - loss: -53043.7903 - binary_accuracy: 0.0367
Epoch 3/3
38697/38697 [==============================] - 0s 9us/step - loss: -53791.6363 - binary_accuracy: 0.0367


In [58]:
x_train.shape

(38697, 484)

In [59]:
x_test = x_test.drop(['HADM_ID'],axis=1)

In [62]:
y_nn_pred = model.predict(x_test)
y_nn_pred

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [9]:
clf_multiregressor.fit(x_train,y_predtrain_new)

MultiOutputRegressor(estimator=LGBMRegressor(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             iterations=1000, learning_rate=0.1,
                                             max_depth=-1, min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             num_leaves=31, objective=None,
                                             random_state=None, reg_alpha=0.0,
                                             reg_lambda=0.0, silent=True,
                                             subsample=1.0,
                                             subsample_for_bin=200000,
  

In [10]:
y_pred_new = clf_multiregressor.predict(x_test)

In [11]:
np.unique(y_pred_new.round())

array([-1., -0.,  1.])

In [12]:
y_combine = np.add(y_pred_new,test_pred)

In [23]:

y_combine_sub = np.subtract(test_pred,y_pred_new)

In [24]:
y_combine_sub  = pd.DataFrame(y_combine_sub)
y_combine_sub

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.089024,0.250961,0.731364,0.479592,0.363976,0.484101,0.525417,0.167312,0.597761,1.111201,-0.003472,0.254758,0.115054,-0.017992,0.053206,0.345565,0.020612,0.476746,0.191181,0.245161
1,-0.015704,0.625114,0.600488,-0.256068,0.158268,0.486627,0.636241,0.194116,0.229766,0.009327,-0.030447,0.094053,0.215548,0.069541,0.061330,0.212783,-0.038751,0.347395,0.227213,0.560036
2,0.136953,0.121940,0.638083,0.587459,0.586738,0.476469,0.636165,0.494035,0.163427,0.268133,0.003852,0.455033,0.231260,0.058612,0.062167,0.227816,0.030377,0.484164,0.467630,0.610517
3,0.109839,0.171064,0.684051,0.796346,0.279918,0.565330,0.946799,0.400425,0.011212,0.476817,0.009218,0.054237,0.370017,0.039760,0.010794,0.324196,-0.033113,0.698318,0.498462,0.579901
4,-0.040466,0.112364,0.857227,0.071119,0.412781,0.450713,0.846847,0.019315,0.166674,0.224935,-0.001844,-0.159370,0.401841,0.043713,0.108154,0.425024,0.024754,0.032917,0.266417,0.624878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6824,0.377572,-0.144161,0.937939,0.570449,0.285378,0.760439,1.016193,0.597300,0.509711,0.719951,0.002696,0.142335,0.278681,0.038350,0.043628,0.652908,0.016492,0.108819,0.527513,0.782148
6825,0.060873,0.182967,1.350453,0.524553,0.629730,0.697863,1.738316,0.716916,0.735561,0.779324,0.004178,0.160486,0.268505,-0.286545,-0.602017,0.628832,0.042157,0.823958,0.516648,-0.040224
6826,-0.002829,0.141023,-0.259838,-0.167407,0.243480,0.164786,0.032320,-0.084057,0.155798,-0.571634,0.001154,-0.074066,-0.003181,0.083161,0.081390,0.158078,-0.089421,0.323387,0.388975,-0.213115
6827,0.004321,0.268889,0.421791,0.179655,0.015012,-0.309269,0.785318,0.089934,0.406110,-0.375524,0.002476,-0.101511,0.243901,0.048683,0.066064,-0.055299,0.029353,0.029469,0.133513,0.322499


In [13]:
y_combine  = pd.DataFrame(y_combine)
#y_combine.to_pickle('./y_combine_nlp_pred-both-2_lgbm-lgbm.pkl')

In [14]:
y_combine.shape

(6829, 20)

In [15]:
y_combine_pred = y_combine.round()  


In [16]:
np.unique(y_combine_pred)

array([-1.,  0.,  1.,  2.])

In [25]:
np.unique(y_combine_sub)

array([-1.01346555, -1.01235571, -1.01164198, ...,  1.77417679,
        1.78040974,  1.82894124])

In [26]:
y_combine_sub = y_combine_sub.round()  #try may not work...lol

In [27]:
y_combine_sub = np.where(y_combine_sub<=0,0,1)

In [22]:
y_pred_new

array([[-0.06823187, -0.17526584,  0.03045973, ...,  0.31961258,
         0.51813888,  0.0788893 ],
       [ 0.02968211, -0.02689267, -0.00202419, ...,  0.25347871,
         0.07784781, -0.15913938],
       [-0.03404724, -0.03054752,  0.01003383, ..., -0.11611315,
        -0.09655913, -0.11608397],
       ...,
       [ 0.00929055, -0.07094428,  0.37896604, ..., -0.10995306,
        -0.05564263,  0.37177738],
       [ 0.01080355, -0.0919711 ,  0.2044044 , ...,  0.21507129,
         0.05539699,  0.2057764 ],
       [-0.03192861, -0.04534699,  0.13537183, ..., -0.04688929,
         0.20660085,  0.08754244]])

In [17]:
y_combine

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.047440,-0.099571,0.792284,-0.076286,0.321183,0.290173,0.703981,0.434097,0.091654,0.443956,0.003472,-0.073582,0.008126,0.052001,-0.053201,0.418202,-0.016278,1.115971,1.227459,0.402940
1,0.043660,0.571328,0.596439,0.462074,0.220906,0.532752,0.898604,0.856832,0.395351,0.623324,0.030447,0.213357,0.262163,-0.057446,-0.061329,0.419696,0.047776,0.854353,0.382909,0.241758
2,0.068858,0.060845,0.658150,0.652253,0.619989,0.323080,0.952889,0.309586,0.468241,-0.154961,-0.003852,0.256149,0.145461,-0.027219,-0.062157,0.173534,0.000420,0.251938,0.274511,0.378349
3,0.156458,0.013731,1.007241,0.862772,0.122024,0.906787,1.045990,1.122620,1.106831,1.447229,-0.009218,0.374337,0.201257,-0.019178,-0.010792,0.598615,0.057052,1.102871,0.796401,1.021490
4,0.068373,0.320736,0.761238,0.372769,0.103552,0.173268,1.048261,0.508220,0.411003,0.139281,0.001844,0.272205,0.227026,-0.036240,-0.108152,0.078820,0.022521,0.479517,0.198260,0.349734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6824,0.371883,0.561611,0.763201,0.744029,0.513966,0.942696,0.922470,1.272640,0.582002,0.285516,-0.002696,0.276850,0.205664,-0.016877,-0.043610,0.634732,0.005516,0.673199,0.721166,0.730419
6825,-0.047107,-0.115683,0.128838,-0.167824,0.048950,-0.046865,0.258239,-0.184704,0.043760,-0.008420,-0.004178,-0.091283,-0.083552,0.326672,0.602024,0.160343,-0.011126,-0.014151,-0.130919,0.736725
6826,0.015752,-0.000865,0.498094,0.252441,0.313211,0.362198,0.494073,0.332953,0.177157,0.641892,-0.001154,0.147396,0.156286,0.025253,-0.081387,0.285925,0.104603,0.103481,0.277689,0.530439
6827,0.025928,0.084947,0.830600,0.098788,0.364313,0.575207,0.596482,0.671237,0.247890,0.676444,-0.002476,0.234900,0.106431,0.001815,-0.066061,0.390604,-0.005444,0.459611,0.244307,0.734052


In [18]:
y_test

,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
22416,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,1,1,0
16012,0,0,0,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0
6206,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
49055,0,0,1,0,0,1,0,1,1,1,0,1,1,0,0,0,0,1,0,1
30123,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44828,0,1,0,1,0,1,1,1,0,0,0,0,1,0,0,1,0,1,0,1
22726,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
58201,0,0,1,1,0,1,1,1,1,1,0,0,0,0,0,1,0,0,0,1
45005,0,0,1,0,0,1,1,0,0,1,0,1,1,1,0,1,0,1,1,0


In [20]:
y_combine_pred = np.where(y_combine_pred<=0,0,1)
y_combine_pred
y_combine_pred_data  = pd.DataFrame(y_combine_pred)

In [21]:
y_combine_pred_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0
1,0,1,1,0,0,1,1,1,0,1,0,0,0,0,0,0,0,1,0,0
2,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,0,1,1,1,1,1,0,0,0,0,0,1,0,1,1,1
4,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6824,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,0,1,1,1
6825,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
6826,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
6827,0,0,1,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1


In [38]:
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
#cm = confusion_matrix(y_test,y_pred) 
#Classification metrics can't handle a mix of multilabel-indicator and unknown targets
print(jaccard_score(y_test,y_combine_pred,average='micro'))
print(jaccard_score(y_test,y_combine_pred,average='macro'))
print(roc_auc_score(y_test,y_combine_pred))
print(hamming_loss(y_test,y_combine_pred))


0.4249304995067707
0.312039559010326
0.620836485993164
0.2347561868501977


In [ ]:
#i have a doubt ,how can we 